# Part 1

Import some of the libraries used

In [20]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

Get the Wiki page and parse it with BeautifulSoup

In [21]:
wiki_toronto = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').content, 'lxml')

Extract postal codes, boroughs and neighborhoods and put them in a DataFrame

In [29]:
def clean_content(content):
    content = content.replace(' / ', ', ')
    content = content.replace('(', '')
    content = content.replace(')', '')
    return content

items = []
for item in wiki_toronto.table.find_all('td'):
    postal_code = item.p.b.text
    if item.p.span.text == "Not assigned":
        continue
    parts = [it for it in item.stripped_strings if it not in "()"]
    borough = parts[1]
    neighborhood = ' '.join(parts[2:])
    
    items.append((postal_code, clean_content(borough), clean_content(neighborhood)))

df_toronto = pd.DataFrame([it for it in items], columns=['PostalCode', 'Borough', 'Neighborhood'])
df_toronto.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District , Ryerson"


In [30]:
df_toronto.shape

(103, 3)

# Part 2

In [31]:
# The code was removed by DSX for sharing.

Get location coordinates for all Postal Codes

In [32]:
locations = []

for item in df_toronto['PostalCode']:
        
    # construct URL to make API call
    url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address="Toronto, ON {}"'.format(API_key, item)

    response = requests.get(url).json() # get response
    
    if response['status'] == 'ZERO_RESULTS':
        print(item)
        continue
    
    geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
    latitude = geographical_data['lat']
    longitude = geographical_data['lng']
    locations.append((item, latitude, longitude))
    
df_locations = pd.DataFrame([it for it in locations], columns=['PostalCode', 'Latitude', 'Longitude'])
df_locations.tail()

,PostalCode,Latitude,Longitude
98,M8X,43.653654,-79.506944
99,M4Y,43.665860,-79.383160
100,M7Y,43.662744,-79.321558
101,M8Y,43.636258,-79.498509
102,M8Z,43.628841,-79.520999


Merge coordinates with the rest of the data

In [34]:
df_data = df_toronto.merge(df_locations, on="PostalCode", how='left')

df_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District , Ryerson",43.657162,-79.378937
